In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
dataset = pd.read_csv("riiid-test-answer-prediction/train.csv")

In [2]:
dataset.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [3]:
dataset.tail(100)

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
101230232,101230232,1262012556,2147482216,1195,0,207,0,1,13000.0,True
101230233,101230233,1262056337,2147482216,375,0,208,0,1,15000.0,True
101230234,101230234,1262083396,2147482216,211,0,209,0,1,19000.0,True
101230235,101230235,1262154833,2147482216,5711,0,210,2,0,18000.0,True
101230236,101230236,1262202756,2147482216,9058,0,211,2,1,21000.0,True
101230237,101230237,1262275724,2147482216,3756,0,212,3,0,19000.0,True
101230238,101230238,1296653480,2147482216,4473,0,213,2,0,40000.0,True
101230239,101230239,1296694380,2147482216,5583,0,214,2,1,28000.0,True
101230240,101230240,1296765032,2147482216,8898,0,215,3,1,25000.0,True
101230241,101230241,1296872450,2147482216,4039,0,216,0,0,14000.0,True


# ................................................................................................................................

## Selecting Only Questions from Dataset

# ...............................................................................................................................

In [24]:
user_question = dataset[dataset['content_type_id']==0]
user_question.shape

(99271300, 10)

In [6]:
user_question.isnull().sum()

row_id                                 0
timestamp                              0
user_id                                0
content_id                             0
content_type_id                        0
task_container_id                      0
user_answer                            0
answered_correctly                     0
prior_question_elapsed_time       392506
prior_question_had_explanation    392506
dtype: int64

In [7]:
user_question['user_id'].nunique()

393656

# ................................................................................................................................

## Selecting Time Steps for RNN

# ...............................................................................................................................

In [4]:
grouped_data = user_question.groupby('user_id').count()
grouped_data.head()

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,46,46,46,46,46,46,46,45,45
124,30,30,30,30,30,30,30,29,29
2746,19,19,19,19,19,19,19,18,18
5382,125,125,125,125,125,125,125,124,124
8623,109,109,109,109,109,109,109,108,108


In [6]:
for i in range(10,32):
    print(i, grouped_data[grouped_data['row_id'] >= i].shape)

10 (391028, 9)
11 (390686, 9)
12 (389703, 9)
13 (382696, 9)
14 (374970, 9)
15 (370859, 9)
16 (368326, 9)
17 (365522, 9)
18 (348385, 9)
19 (343296, 9)
20 (337987, 9)
21 (332232, 9)
22 (324714, 9)
23 (321097, 9)
24 (318775, 9)
25 (317157, 9)
26 (315695, 9)
27 (313861, 9)
28 (303988, 9)
29 (300949, 9)
30 (297146, 9)
31 (234724, 9)


#### We will go ahead with 30 Time Steps

In [9]:
users = grouped_data[grouped_data['row_id'] >= 30].index
selected_user = user_question[user_question['user_id'].apply(lambda x: x in users)]
selected_user.shape

(97439225, 10)

# ................................................................................................................................

## Extracting Answered Correctly Column for each User

# ...............................................................................................................................

In [33]:
# TRY 1

x = []
y = []
for user in users:
    check = selected_user[selected_user["user_id"]==user]
    check = check[-30:]
    val = list(check["answered_correctly"].values)
    x.append(val[:-1])
    y.append(val[-1])

KeyboardInterrupt: 

In [36]:
len(x)

9914

In [ ]:
# TRY 2

dataset = dataset.to_numpy()

dataset1 = dataset[::-1]
x = []
y = []
for user in users[5:6]:
    check = dataset[:,2]
    check = check[-30:]
    val = list(check["answered_correctly"].values)
    x.append(val[:-1])
    y.append(val[-1])

In [ ]:
# TRY 3

dataset = dataset[dataset['user_id'].apply(lambda x: x in users)]
dataset = dataset.iloc[::-1]
grouped_data = dataset.groupby('user_id')

x = []
y = []
for key,group in grouped_data:
    group = group.values[:30,7]
    group = group[::-1]
    x.append(group[:29])
    y.append(group[29])

In [ ]:
# Exporting Data into a file

import csv

file = open('x.csv', 'a+', newline ='')
with file:
    write = csv.writer(file) 
    write.writerows(x)

with open('y.csv', 'w') as filehandle:
    for listitem in y:
        filehandle.write('%s\n' % listitem)

# ................................................................................................................................

## Extracting Prior Question Had Explanation Column for each User

# ...............................................................................................................................

In [ ]:
dataset['prior_question_had_explanation'] = dataset['prior_question_had_explanation'].fillna(False)

def func(val):
    if val == False:
        return 0
    elif val == True:
        return 1

dataset['prior_question_had_explanation'] = dataset['prior_question_had_explanation'].apply(func)



dataset = dataset[dataset['user_id'].apply(lambda x: x in users)]
dataset = dataset.iloc[::-1]
grouped_data = dataset.groupby('user_id')

x = []
y = []
for key,group in grouped_data:
    print(key)
    group = group.values[:30,9]
    group = group[::-1]
    x.append(group[:29])
    y.append(group[29])

# ................................................................................................................................

## Extracting Part to which each question belong Column

# ...............................................................................................................................

In [1]:
import pandas as pd
import numpy as np
questions = pd.read_csv("riiid-test-answer-prediction/questions.csv")

In [2]:
questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [3]:
dict_ques = {}
for row in questions.iterrows():
    dict_ques[row[1][0]] = row[1][3]

In [4]:
dict_ques

{0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 1,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 1,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 1,
 44: 1,
 45: 1,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 1,
 55: 1,
 56: 1,
 57: 1,
 58: 1,
 59: 1,
 60: 1,
 61: 1,
 62: 1,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 1,
 68: 1,
 69: 1,
 70: 1,
 71: 1,
 72: 1,
 73: 1,
 74: 1,
 75: 1,
 76: 1,
 77: 1,
 78: 1,
 79: 1,
 80: 1,
 81: 1,
 82: 1,
 83: 1,
 84: 1,
 85: 1,
 86: 1,
 87: 1,
 88: 1,
 89: 1,
 90: 1,
 91: 1,
 92: 1,
 93: 1,
 94: 1,
 95: 1,
 96: 1,
 97: 1,
 98: 1,
 99: 1,
 100: 1,
 101: 1,
 102: 1,
 103: 1,
 104: 1,
 105: 1,
 106: 1,
 107: 1,
 108: 1,
 109: 1,
 110: 1,
 111: 1,
 112: 1,
 113: 1,
 114: 1,
 115: 1,
 116: 1,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 122: 1,
 12

In [5]:
dataset = pd.read_csv("riiid-test-answer-prediction/train.csv")

dataset = dataset[dataset['content_type_id']==0]

In [6]:
def func(var):
    return dict_ques[var]

dataset['part'] = dataset['content_id'].apply(func)

In [7]:
dataset.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,part
0,0,0,115,5692,0,1,3,1,NaN,NaN,5
1,1,56943,115,5716,0,2,2,1,37000.0,False,5
2,2,118363,115,128,0,0,0,1,55000.0,False,1
3,3,131167,115,7860,0,3,0,1,19000.0,False,1
4,4,137965,115,7922,0,4,1,1,11000.0,False,1


In [8]:
dataset['part'].value_counts()

5    40908153
2    18743404
6    10501472
3     8639907
4     8067676
1     7454570
7     4956118
Name: part, dtype: int64

In [9]:
grouped_data = dataset.groupby('user_id').count()
users = grouped_data[grouped_data['row_id'] >= 30].index

In [10]:
data = dataset[['row_id','timestamp','user_id','part']].values
data[0]

array([  0,   0, 115,   5], dtype=int64)

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer([('encoder', OneHotEncoder(), [3])], remainder='passthrough')
data_new = np.array(ct.fit_transform(data), dtype=np.float)
data_new[0]

array([  0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0., 115.])

In [12]:
data_new = data_new[:,1:]
data_new[0]

array([  0.,   0.,   0.,   1.,   0.,   0.,   0.,   0., 115.])

In [13]:
dataset = pd.DataFrame(data_new, columns = ['x1','x2','x3','x4','x5','x6','row_id','timestamp','user_id'])
dataset.head(60)

,x1,x2,x3,x4,x5,x6,row_id,timestamp,user_id
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,115.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,56943.0,115.0
2,0.0,0.0,0.0,0.0,0.0,0.0,2.0,118363.0,115.0
3,0.0,0.0,0.0,0.0,0.0,0.0,3.0,131167.0,115.0
4,0.0,0.0,0.0,0.0,0.0,0.0,4.0,137965.0,115.0
5,0.0,0.0,0.0,0.0,0.0,0.0,5.0,157063.0,115.0
6,0.0,0.0,0.0,0.0,0.0,0.0,6.0,176092.0,115.0
7,0.0,0.0,0.0,0.0,0.0,0.0,7.0,194190.0,115.0
8,0.0,0.0,0.0,0.0,0.0,0.0,8.0,212463.0,115.0
9,0.0,0.0,0.0,0.0,0.0,0.0,9.0,230983.0,115.0


In [14]:
dataset = dataset[dataset['user_id'].apply(lambda x: x in users)]
dataset = dataset.iloc[::-1]

In [15]:
grouped_data = dataset.groupby('user_id')

x = []
y = []
for key,group in grouped_data:
    group = group.values[:30,:6]
    group = group[::-1]
    x.append(group[:29])
    y.append(group[29])

In [16]:
x[:5]

[array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.]]),
 array([[0.,

In [18]:
new_x = np.array(x)
new_x.shape

(297146, 29, 6)

In [19]:
try_x = new_x.reshape(297146, 174)
np.savetxt("x_part.txt", try_x)